In [1]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm # show progress bar

In [4]:
for i in tqdm(range(100)):
    time.sleep(1)
    

 24%|██▍       | 24/100 [00:24<01:16,  1.01s/it]


KeyboardInterrupt: 